## FINAL CAPSTONE PROJECT
## PERSONAL PROTECTIVE EQUIPMENTS STORE

This is a store that will works with database to keep track of the item's price and stock. In the end of purchasement, customer can print out reciept in the form of simple file-stream that will be saved locally. <br>
This is a local application and for education purpose only. This is not a real-world utility application.

## Import Library

In [1]:
from IPython.display import clear_output
import pyodbc
import pdb
import datetime

## Database setup

In [2]:
dbConn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=LAPTOP-LM5VM6EL\SQLEXPRESS;'
    'Database=PythonStore;'
    'Trusted_Connection=yes;'
)

dbCursor = dbConn.cursor()

dbCursor.execute("SELECT * FROM Items")

In [3]:
itemList = [item for item in dbCursor]

In [4]:
for item in itemList:
    print(item)

('Hand Sanitizer', 101, 30000, 15)
('Q-Mask', 102, 45000, 20)
('Easy-Mask', 103, 60000, 20)
('Face Shield OC', 104, 60000, 15)
('Thermo Gun STD', 105, 650000, 15)
('Thermo Gun LSR', 106, 750000, 10)
('Thermo Gun FLT', 107, 700000, 10)
('Thermo Gun VCE', 108, 800000, 8)
('Mdc Gloves NTRL', 109, 5000, 50)
('Mdc Mask STD', 110, 4000, 50)


In [76]:
itemList[0][1]

101

## Classes

In [5]:
class Customer():
    def __init__(self, name):
        '''
        #Later on, this is going to be list of dictionaries where the format: 
        #[
        #{itemID1 : {"itemName1":"name1", "itemPrice1":price1, "itemBuyQuant1":quant1, "itemSubtotal1":subtotal1}},
        #{itemID2 : {"itemName2":"name2", "itemPrice2":price2, "itemBuyQuant2":quant2, "itemSubTotal2":subtotal2}},
        #ETC. . .
        #]
        '''
        self.name = name
        self.cart = {}
        self.totalPrice = 0
        
    def __str__(self):
        tempCart = ""
        for item in self.cart.items():
            tempCart += item + "\n"
            
        customer = f"Hello my name is {self.name}\n My item in cart: {tempCart}\nThe total price: {self.totalPrice}"
        
        return customer
    
    def addToChart(self, item, itemID):
        #We expect item to be a dictionary that has dictionary as a pair value. 
        if itemID in self.cart.keys():
            self.cart[itemID]["itemQuant"] += item[itemID]["itemQuant"]
            self.cart[itemID]["subTotal"] += item[itemID]["subTotal"]
        else:
            self.cart.update(item)
            
    def removeFromChart(self, removeQuant, itemID):
        currentItemQuant = self.cart[itemID]["itemQuant"]
        if removeQuant == currentItemQuant:
            del self.cart[itemID]
        else:
            newQuant = currentItemQuant - removeQuant
            self.cart[itemID]["itemQuant"] = newQuant
            
            itemPrice = self.cart[itemID]["itemPrice"]
            
            newSubTotal = itemPrice * newQuant
            self.cart[itemID]["subTotal"] = newSubTotal

## Functions

In [6]:
def dispItemName(itemList):
    template = "{0:^2} | {1:^15} | {3:<8}"
    print(template.format("NO", "ITEM LIST","_", "ITEM PRICE"))
    print("================================")
    for index, item in enumerate(itemList, start=1):
        print(template.format(index, *item))
        print("================================")
    print(f"{len(itemList)+1} | Exit")

In [7]:
dispItemName(itemList)

NO |    ITEM LIST    | ITEM PRICE
1  | Hand Sanitizer  | 30000   
2  |     Q-Mask      | 45000   
3  |    Easy-Mask    | 60000   
4  | Face Shield OC  | 60000   
5  | Thermo Gun STD  | 650000  
6  | Thermo Gun LSR  | 750000  
7  | Thermo Gun FLT  | 700000  
8  | Thermo Gun VCE  | 800000  
9  | Mdc Gloves NTRL | 5000    
10 |  Mdc Mask STD   | 4000    
11 | Exit


In [8]:
def askToBuy(itemList):
    while True:
        try:
            while True:
                clear_output()
                dispItemName(itemList)
                userItemPick = int(input("What would you like to buy? "))
                if userItemPick > len(itemList)+1:
                    clear_output()
                    input("Sorry, your choice exceeding the limit. Enter anything to re-select items. . .")
                else:
                    break
        except:
            clear_output()
            input("Sorry, that was invalid input. Enter anything to re-select items. . .")
            continue
        else:
            return userItemPick-1

In [9]:
def askQuantity(itemNumber):
    itemName = itemList[itemNumber][0]
    while True:
        try:
            clear_output()
            itemQuant = abs(int(input(f"{itemName} Quantity: ")))
        except:
            clear_output()
            input("Sorry, invalid input. Enter anything to re-select. . .")
        else:
            break
    return itemQuant

In [10]:
def purchaseAgain():
    while True:
        clear_output()
        userChoice = input("Do you want to purchase another item? [Y/N]")
        if userChoice[0].lower() != "y" and userChoice[0].lower() != "n":
            continue
        break
    return userChoice[0].lower()

In [11]:
def viewCart(cartItemList):
    '''
    Expected parameters:
    -cartItemList:
        #All values inside the dictionary of user's cart. 
        #Use dict.values() to get all of the values. 
    '''
    clear_output()
    print("Your groceries: ")
    print("{:^3}|{:^20}|{:^10}|{:^10}|{:^10}".format('NO.', 'ITEM NAME', 'PRICE', 'QUANTITY', "SUB-TOTAL"))
    print("================================================================")
    for index, item in enumerate(cartItemList, start=1):
        print(
            f"{index:<3}|{item['itemName']:<20}|{item['itemPrice']:<10}|{item['itemQuant']:<10}|{item['subTotal']}"
        )
        print("================================================================")

In [12]:
def modCart():
    try:
        userChoice = abs(int(input("What would you like to do with your cart?\n[1]. Add item\n[2]. Remove item\n[3]. Exit\n")))
        if userChoice > 3 or userChoice < 1:
            clear_output()
            input("Sorry, input out of range. Please input number according to available menu. . .")
            return False
    except:
        clear_output()
        input("Sorry, invalid input. Please provide number only. . .")
        return False
    else:
        return userChoice

In [13]:
def removeItem(userCart, customer):
    #userCart is a dictionary items that contain tuples where each tuples has length of 2. 
    #Index 0 is the key, index 1 is a dictionary of values.
    tupOfItems = list(userCart.items())
    while True:
        try:
            while True:
                viewCart(userCart.values())
                userChoice = abs(int(input("Which item would you like to remove?")))-1
                if userChoice+1 > len(userCart) or userChoice < 0:
                    clear_output()
                    input("Sorry, input out of bound. Please provide an available input. . .")
                    continue
                break
        except:
            clear_output()
            input("Sorry, input was not a number. Please provide a number. . .")
            continue
        else:
            break
    
    clear_output()
    selectedName = tupOfItems[userChoice][1]["itemName"]
    selectedItemQuant = tupOfItems[userChoice][1]["itemQuant"]
    
    while True:
        try:
            while True:
                removeQuant = abs(int(input(f"How many {selectedName} you want to remove?\nCurrently in chart: {selectedItemQuant}\n")))
                if removeQuant > selectedItemQuant:
                    clear_output()
                    input(f"Fail to remove item. Requested ammount exceeding the current ammount.\nPlease input anything to re-input\nCurrent ammount: {selectedItemQuant}")
                    continue
                break
        except:
            clear_output()
            input("Sorry, input was not a number. Please provide a number. . .")
            continue
        else:
            break
    
    clear_output()
    
    itemID = getKey(userCart, selectedName)
    
    customer.removeFromChart(removeQuant, itemID)

In [14]:
def getKey(dictionary, value):
    #The dictionary here has integer as a key and another dictionary as value.
    for key in dictionary.keys():
        if dictionary[key]["itemName"] == value:
            return key
    return f"No key that has value {value}"

In [20]:
sampleList = {
    101:{'itemName': 'Thermo Gun STD', 'itemPrice': 650000, 'itemQuant': 1, 'subTotal': 650000},
    102:{'itemName': 'Mdc Mask STD', 'itemPrice': 4000, 'itemQuant': 2, 'subTotal': 8000},
    103:{'itemName': 'Thermo Gun FLT', 'itemPrice': 700000, 'itemQuant': 2, 'subTotal': 1400000}
}

In [16]:
def askPayment(totalPrice):
    try:
        userPay = abs(int(input("User pay: ")))
        if userPay < totalPrice:
            clear_output()
            input("Not sufficient! Please input equal or more than the total price.\nEnter anything to re-input")
            return False
    except:
        input("Sorry, input is not valid. Please input numbers only. . .")
        return False
    else:
        return userPay

In [39]:
def storeReciept(userCartValues, total, pay, change):
    '''
    #Purpose:
        -This function will return a very large string, which is the reciept, to be stored in .txt file. 
    #Parameters:
        -userCartValues:
            ~Type: list of values of a dictionary
            ~Expected to be a list of values generated from dictionary of user's cart. 
            ~Pass the argument as dict.values()
        -total:
            ~Type: integer
            ~The total grocery price from user
        -pay:
            ~Type: integer
            ~The total ammount that user actually pay to the casheer
        -change:
            ~Type: integer
            ~The result from the 'pay' subtracted with total to get the ammount of change
    '''
    finalString = ""
    finalString += "{:^3}|{:^20}|{:^10}|{:^10}|{:^10}".format('NO.', 'ITEM NAME', 'PRICE', 'QUANTITY', "SUB-TOTAL") + "\n"
    finalString += "================================================================" + "\n"
    for index, item in enumerate(userCartValues, start=1):
        finalString += f"{index:<3}|{item['itemName']:<20}|{item['itemPrice']:<10}|{item['itemQuant']:<10}|{item['subTotal']}" + "\n"
        finalString += "================================================================" + "\n"
        
    finalString += f"\t\t\t\t\tTOTAL: {str(total)}" + "\n"
    finalString += f"\t\t\t\t\tPAY:   {str(pay)}" + "\n"
    finalString += f"\t\t\t\t\tCHANGE: {str(change)}" + "\n"
    
    return finalString

In [40]:
result = storeReciept(sampleList.values(), 200000, 250000, 50000)

In [41]:
print(result)

NO.|     ITEM NAME      |  PRICE   | QUANTITY |SUB-TOTAL 
1  |Thermo Gun STD      |650000    |1         |650000
2  |Mdc Mask STD        |4000      |2         |8000
3  |Thermo Gun FLT      |700000    |2         |1400000
					TOTAL: 200000
					PAY:   250000
					CHANGE: 50000



## Global Variables

In [18]:
aboutStore = '''
* * * * * 
Our store specifically sells PES Equipments with great-guaranteed quality.
We sell items starting from basic hand-sanitizer until the latest thermogun model.
Our store is located in the middle of the town at: 
\txxth Avenue
\txxth Larenggam street
\tBuilding number 5

We prioritize customer's experience along with the quality of our products and services. 
Our product is sealed neatly to prevent contamination from viruses and bacteria for our product.

So what are you waiting for? Come down to our store and grab your utilities now.
* * * * *
'''

## Driver Code

In [82]:
storeOn = True

customer = Customer("Henry")

while storeOn:
    
    clear_output()
    print("Hello! Welcome to Personal Protective Equipment Store!")
    mainMenu = input("Please choose one of these menu:\n[1]. Buy\n[2]. View Chart\n[3]. Payment\n[4]. About Store\n[5]. Exit\n")
    
    if mainMenu == "1":
        #Purchase Menu
        while True:
            itemNumber = askToBuy(itemList)
            if itemNumber == len(itemList):
                break

            itemQuantity = askQuantity(itemNumber)

            itemName = itemList[itemNumber][0]
            itemID = itemList[itemNumber][1]
            itemPrice = itemList[itemNumber][2]
            itemSubTotal = itemPrice * itemQuantity

            itemTempCart = {
                itemID : {"itemName":itemName, "itemPrice":itemPrice, "itemQuant":itemQuantity, "subTotal":itemSubTotal}
            }

            customer.addToChart(itemTempCart, itemID)
            
            if purchaseAgain() == "y":
                continue
            break
            
    elif mainMenu == "2":
        #View Chart Menu
        while True:
            viewCart(customer.cart.values())
            
            itemTotalPrice = 0
            for subTotal in customer.cart.values():
                itemTotalPrice += subTotal['subTotal']
                
            print(f"\t\t\t\t\tTOTAL: {itemTotalPrice}")
            
            userMod = modCart()
            if userMod == False:
                continue
            break
        
        if userMod == 1:
            #Add item
            clear_output()
            input("Please refer to the buy menu. . .")
        elif userMod == 2:
            #Remove item
            clear_output()
            removeItem(customer.cart, customer)
            clear_output()
            input("Item Removed successfuly!")
        else:
            #Exit view cart menu
            continue

        
    elif mainMenu == "3":
        #Payment menu
        
        itemTotalPrice = 0
        for subTotal in customer.cart.values():
            itemTotalPrice += subTotal['subTotal']
                
        while True:
            viewCart(customer.cart.values())
            print(f"\t\t\t\t\tTOTAL: {itemTotalPrice}")
            pay = askPayment(itemTotalPrice)
            if pay == False:
                continue
            break
                
        changeMoney = pay - itemTotalPrice
        viewCart(customer.cart.values())
        print(f"\t\t\t\t\tTOTAL: {itemTotalPrice}")
        print(f"\t\t\t\t\tPAYMENT: {pay}")
        print(f"\t\t\t\t\tCHANGE: {changeMoney}")
        printReciept = input("Do you want to print reciept?\n[Press \"Y\" if you want to continue. . .]")
        if printReciept[0].lower() == "y":
            currentDateTime = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M")
            fileName = f"{customer.name}_{currentDateTime}"
            
            with open(f"reciept\\{fileName}.txt", mode="w") as file:
                temp = customer.cart.values()
                reciept = storeReciept(temp, itemTotalPrice, pay, changeMoney)
                file.write(reciept)
            
            clear_output()
            input("All Done! Press anything to quit. . .")
        break
        
    
    elif mainMenu == "4":
        #About Store Menu
        clear_output()
        print(aboutStore)
        input("Input anything to go back to previous menu. . .")
        
    elif mainMenu == "5":
        #Exit menu
        break
    else:
        continue
        
clear_output()
print("Thank you for using our store!\nPlease come again. . .")

Thank you for using our store!
Please come again. . .


# IMPORTANT!!
# RUN THE CELL BELOW IF YOU WANT TO EXIT!
# ELSE, SQL SERVER MANAGEMENT STUDIO CAN'T BE USED EXTERNALLY

In [83]:
dbCursor.close()
dbConn.close()

In [79]:
sampleDate = datetime.datetime.now().strftime("%Y-%m-%d, %H-%M")

In [80]:
sampleVar = "sampleFileName"
with open(f"reciept\\{sampleDate}.txt", mode="w") as file:
    file.write("Hello world!\nSample File and string")

In [81]:
print(sampleDate)

2020-07-10, 23-27
